In [3]:
import cv2
img = cv2.imread('shape.png', 0)
img2=cv2.resize(img, (500, 500))
cv2.imwrite("shape.png", img2)           
           

True

In [11]:
import cv2
img = cv2.imread('SI.jpg')
img2=cv2.resize(img, (500, 500))
cv2.imwrite("SI.jpg", img2)       

True

In [11]:
#Chapter6_1.1 라벨링 알고리즘


def labelset(img, xs, ys, label):
    height, width = img.shape
    img[ys, xs] = label
    while True:
        cnt = 0
        for y in range(1, height - 1):
            for x in range(1, width - 1):
                if img [y, x+1] ==255:
                    img [y,x+1 ] = label; cnt=cnt+1
                if img [y-1, x+1] ==255:
                    img [y-1,x+1 ] = label; cnt=cnt+1
                if img [y-1, x] ==255:
                    img [y-1,x ] = label; cnt=cnt+1
                if img [y-1, x-1] ==255:
                    img [y-1,x-1 ] = label; cnt=cnt+1
                if img [y, x-1] ==255:
                    img [y,x-1 ] = label; cnt=cnt+1   
                if img [y+1, x-1] ==255:
                    img [y+1,x-1 ] = label; cnt=cnt+1
                if img [y+1, x] ==255:
                    img [y+1,x ] = label; cnt=cnt+1
                if img [y+1, x+1] ==255:
                    img [y+1,x+1 ] = label; cnt=cnt+1
                # ... (다른 if 문들은 생략)
        if cnt == 0:
            return (0, img)
    return (1,img)
 

 
def labeling(img_in, base=100):
    height, width = img_in.shape
    img_label = img_in.copy()

    label = base  # 수정: Base가 base로 변경되었습니다.

    for y in range(1, height - 1):
        for x in range(1, width - 1):
            if img_label[y, x] == 255:
                if label >= 255:
                    print("Error! Too many labels")
                    return -1

                _, img_label = labelset(img_label, x, y, label)  # 수정: labeltest가 labelset으로 변경되었습니다.
                label = label + 1
    cnt = label - base  # 수정: Base가 base로 변경되었습니다.
    return img_label, cnt


import cv2
img = cv2.imread('shape.png', 0)
_, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

labeled_img, cnt = labeling(bin_img)
cv2.imshow('image', img)
cv2.imshow("Labeling", labeled_img)
cv2.waitKey(0)
cv2.destroyAllWindows()






In [12]:
#Chapter6_1.2 connectedComponents() 함수 이용 쉽게 라벨링

import cv2
import numpy as np

img= cv2.imread('shape.png',0)
_, bin_img = cv2.threshold(img,128,255,cv2.THRESH_BINARY)
cnt, labeled_img = cv2.connectedComponents(bin_img)

print(labeled_img[80,80],end="  ")
print(labeled_img[69,173],end="  ")
print(labeled_img[160,190],end="  ")
print(labeled_img.dtype)

labeled_img = labeled_img *50
cv2.imshow("Labeling",labeled_img.astype(np.uint8))
cv2.imwrite("Labeled_img.jpg", labeled_img)      

cv2.waitKey(0)
cv2.destroyAllWindows()


1  0  0  int32


In [13]:
#Chapter6_3.3 이미지에서 객체 탐지하기


import numpy as np

CLASSES = ["background","aeroplane","bicycle","bird","boat","bottle","bus","car","cat","chair","cow","diningtable","dog","horse","motorbike","person","pottedplant","sheep","sofa","train","tvmonitor"]
COLOR = np.random.uniform(0,255,size=(len(CLASSES),3))

import cv2

net = cv2.dnn.readNetFromCaffe("deploy.prototxt","mobilenet_iter_73000.caffemodel")

image = cv2.imread ("work.jpg")
#image = cv2.imread ("SI.jpg")
(h,w) = image.shape[:2]

blob = cv2.dnn.blobFromImage (cv2.resize(image,(300,300)),0.007843,(300,300),127.5)

net.setInput(blob)
detections = net.forward()


#loop over the detections

for i in np.arange(0,detections.shape[2]):
    #탐지 결과에서 신뢰도 추출
    confidence = detections[0,0,i,2]
    if confidence > 0.2 : #confidence 20% 이상만 처리 
        #클래스 레이블의 인덱스와 경계상자 추출
        idx = int(detections[0,0,i,1])
        box = detections[0,0,i,3:7] * np.array([w,h,w,h])
        (startX,startY,endX,endY) = box.astype("int")
        #탐지 결과 표시
        label = "{}:{:.2f}%".format(CLASSES[idx],confidence * 100)

        print("{}".format(label))
        cv2.rectangle(image,(startX,startY),(endX,endY),COLOR[idx],2)



cv2.imshow("Output",image)
cv2.waitKey(0)
cv2.destroyAllWindows()


bottle:99.76%
bottle:33.40%


In [6]:
#Chapter6_3.4 카메라를 이용한 실시간 객체 탐지하기


import numpy as np

CLASSES = ["background","aeroplane","bicycle","bird","boat","bottle","bus","car","cat","chair","cow","diningtable","dog","horse","motorbike","person","pottedplant","sheep","sofa","train","tvmonitor"]
COLORS = np.random.uniform(0,255,size=(len(CLASSES),3))

import cv2
net = cv2.dnn.readNetFromCaffe("deploy.prototxt","mobilenet_iter_73000.caffemodel")

cap = cv2.VideoCapture(0)

if cap.isOpened():
    delay = int(1000/cap.get(cv2.CAP_PROP_FPS))

    while True:
        ret, image = cap.read()
        if ret:
            (h,w) = image.shape[:2]
            blob = cv2.dnn.blobFromImage(cv2.resize(image,(300,300)),0.007843,(300,300),127.5)
            # 탐색 수행
            net.setInput(blob)
            detections=net.forward()

            #탐지 결과 반복
            for i in np.arange(0,detections.shape[2]):
                #탐지 결과에서 신뢰도 추출
                confidence = detections[0,0,i,2]
                if confidence > 0.2 : #confidence 20% 이상만 처리 
                    idx = int(detections[0,0,i,1])
                    box = detections[0,0,i,3:7] * np.array([w,h,w,h])
                    (startX,startY,endX,endY) = box.astype("int")
                    #탐지 결과 표시
                    label = "{}:{:.2f}%".format(CLASSES[idx],confidence * 100)
                    cv2.rectangle(image,(startX,startY),(endX,endY),COLOR[idx],2)
                    y=startY-15 if startY-15 >15 else startY+15
                    cv2.putText(image,label,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.5,COLORS[idx],2)

        #윈도우로 프레임 출력
        cv2.imshow("Detection",image)
        key = cv2.waitKey(delay) & 0xFF
        if key == ord("q") : #q가 눌리면 프레임 종료
            break
else: 
    print("File not opened")

print("{}".format(label))
#print("{}".format(label))


cap.release()
cv2.destroyAllWindows()


person:98.05%
